### 1. Sorting Images

In [1]:
import os, shutil
base_dir = 'C:/Users/user/PycharmProjects/ML_2022/Task_6/dogs-vs-cats'
original_data_dir = os.path.join(base_dir, 'train')

train_cats_dir = os.path.join(original_data_dir, 'cats')
train_dogs_dir = os.path.join(original_data_dir, 'dogs')

os.mkdir(train_cats_dir)
os.mkdir(train_dogs_dir)


FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'C:/Users/user/PycharmProjects/ML_2022/Task_6/dogs-vs-cats\\train\\cats'

In [2]:
fnames = ['cat.{}.jpg'.format(i) for i in range(12500)]
for fname in fnames:
    src = os.path.join(original_data_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copy(src, dst)

fnames = ['dog.{}.jpg'.format(i) for i in range(12500)]
for fname in fnames:
    src = os.path.join(original_data_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copy(src, dst)

print("Finished")

KeyboardInterrupt: 

### 2. Loading Datasets

In [7]:
import torch
import PIL
import numpy as np
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

batch_size = 4

test_files = [f'{i}.jpg' for i in range(1, 12500)]

class CustomDataset(Dataset):    # Define new class CustomDataset for test datasets
    def __init__(self, files, root, transform=None):
        self.files = files
        self.root = root
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        img = PIL.Image.open(os.path.join(self.root, self.files[index]))

        if self.transform:
            img = self.transform(img)

        return img, self.files[index]




# Using ImageFolder with image folders of different classes (i.e., folders cats and dogs) for train sets

train_data = datasets.ImageFolder(root="./train1", transform = preprocess)
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

test_data = CustomDataset(test_files, root='./test1', transform = preprocess)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

print('Finished')
print(train_data[0])

Finished
(tensor([[[ 2.1119,  2.1290,  2.1290,  ...,  2.1975,  2.1975,  2.2147],
         [ 2.1119,  2.1290,  2.1290,  ...,  2.1975,  2.1975,  2.2147],
         [ 2.0948,  2.1119,  2.1119,  ...,  2.1975,  2.1975,  2.2147],
         ...,
         [-0.7479, -0.7650, -0.7822,  ..., -1.7240, -1.7240, -1.7069],
         [-0.7993, -0.7993, -0.7993,  ..., -1.7412, -1.7240, -1.7240],
         [-0.8335, -0.8164, -0.7993,  ..., -1.7069, -1.7412, -1.7754]],

        [[ 1.5357,  1.5532,  1.5532,  ...,  1.9209,  1.9209,  1.9384],
         [ 1.5357,  1.5532,  1.5532,  ...,  1.9209,  1.9209,  1.9384],
         [ 1.5182,  1.5357,  1.5357,  ...,  1.9209,  1.9209,  1.9384],
         ...,
         [-1.0728, -1.0903, -1.1078,  ..., -1.7206, -1.7206, -1.6506],
         [-1.1253, -1.1253, -1.1253,  ..., -1.7381, -1.7206, -1.6681],
         [-1.1604, -1.1429, -1.1253,  ..., -1.7206, -1.7381, -1.7206]],

        [[ 0.3742,  0.3916,  0.3916,  ...,  0.8274,  0.8274,  0.8448],
         [ 0.3742,  0.3916,  0.3916

### 3. Define a Convolutional Neural Network

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(44944, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Linear(10, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

net1 = Net()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
net1.to(device)


cuda:0


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=2, bias=True)
)

### 4. Define a Loss function and optimizer

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net1.parameters(), lr=0.001, momentum=0.9)

### 5. Train the network

In [5]:
epochs = 2

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]

        inputs, labels = data[0].to(device), data[1].to(device)
        print(inputs, labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 1000:.3f}')
            running_loss = 0.0

print('Finished Training')

tensor([[[[ 1.0673,  1.0673,  1.0844,  ...,  1.0331,  1.0331,  1.0159],
          [ 1.0673,  1.0673,  1.0673,  ...,  1.0159,  1.0159,  0.9988],
          [ 1.1358,  1.1358,  1.1187,  ...,  0.9474,  0.9474,  0.9474],
          ...,
          [ 2.1462,  2.1462,  2.1462,  ...,  0.4508,  0.3309,  0.1939],
          [ 2.2318,  2.2318,  2.2318,  ..., -0.2513, -0.3883, -0.4911],
          [ 2.2489,  2.2489,  2.2489,  ..., -0.5767, -0.5082, -0.4054]],

         [[ 1.2731,  1.2731,  1.2906,  ...,  1.1856,  1.1856,  1.1681],
          [ 1.2731,  1.2731,  1.2731,  ...,  1.1681,  1.1681,  1.1506],
          [ 1.3431,  1.3431,  1.3256,  ...,  1.0980,  1.0980,  1.0980],
          ...,
          [ 2.3235,  2.3235,  2.3235,  ...,  0.5203,  0.3978,  0.2577],
          [ 2.4111,  2.4111,  2.4111,  ..., -0.1800, -0.3200, -0.4251],
          [ 2.4286,  2.4286,  2.4286,  ..., -0.4776, -0.4076, -0.3025]],

         [[ 1.5768,  1.5768,  1.5942,  ...,  1.4025,  1.4025,  1.3851],
          [ 1.5768,  1.5768,  

KeyboardInterrupt: 

In [6]:
PATH = './cats_vs_dogs.pth'

### 6. Test the network

In [7]:
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [50]:
import matplotlib.pyplot as plt
import numpy as np

test_files = [f'{i}.jpg' for i in range(1, 12500)]
test_data = CustomDataset(test_files, root='./test1', transform = preprocess)

print(len(test_data))

testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

predicted = list()
for i, data in enumerate(testloader):
    images, labels = data
    outputs = net(images)
    _, predicted_temp = torch.max(outputs, 1)
    predicted += [int(i) for i in predicted_temp]


print(predicted)


12499
[1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 

In [63]:

indices = [i for i in range(1, 12500)]
lines = ['id,label\n'] + [str(m) + ',' + str(n) +'\n' for m,n in zip(indices, predicted)]
print(lines)
with open('submission.csv', 'w') as f:
    f.writelines(lines)


['id,label\n', '1,1\n', '2,1\n', '3,0\n', '4,1\n', '5,0\n', '6,0\n', '7,0\n', '8,0\n', '9,1\n', '10,0\n', '11,0\n', '12,1\n', '13,1\n', '14,0\n', '15,0\n', '16,0\n', '17,1\n', '18,1\n', '19,0\n', '20,0\n', '21,1\n', '22,0\n', '23,1\n', '24,1\n', '25,1\n', '26,0\n', '27,1\n', '28,0\n', '29,0\n', '30,1\n', '31,0\n', '32,0\n', '33,1\n', '34,0\n', '35,0\n', '36,0\n', '37,0\n', '38,0\n', '39,1\n', '40,0\n', '41,1\n', '42,1\n', '43,0\n', '44,1\n', '45,0\n', '46,0\n', '47,0\n', '48,1\n', '49,1\n', '50,0\n', '51,0\n', '52,0\n', '53,0\n', '54,0\n', '55,0\n', '56,0\n', '57,1\n', '58,0\n', '59,1\n', '60,1\n', '61,0\n', '62,0\n', '63,1\n', '64,1\n', '65,1\n', '66,1\n', '67,1\n', '68,0\n', '69,1\n', '70,1\n', '71,1\n', '72,1\n', '73,1\n', '74,1\n', '75,0\n', '76,1\n', '77,1\n', '78,1\n', '79,0\n', '80,0\n', '81,0\n', '82,1\n', '83,1\n', '84,0\n', '85,1\n', '86,0\n', '87,0\n', '88,1\n', '89,0\n', '90,1\n', '91,0\n', '92,1\n', '93,0\n', '94,1\n', '95,1\n', '96,0\n', '97,1\n', '98,1\n', '99,0\n', '100